## Python Notebook to train XGBoost model on both the original and augmented dataset

Here, we use the xgboost package but use the classifier built in the package that is built to integrate with sklearn at the cost of some functionality.
[Source](https://www.datacamp.com/tutorial/xgboost-in-python)

In [9]:
#Import necessary modules
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
import nltk
import xgboost as xgb

## Configure NLTK if applicable

In [ ]:
# nltk.download('averaged_perceptron_tagger')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('vader_lexicon')

In [97]:
# obtained from https://gist.github.com/susanli2016/d35def30b99f0e2f56c0e01e19ad0878
def gettop_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0)
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [98]:
def get_len(row):
    return len(row['Text'])

def get_len_aug(row):
    return len(row['text'])

In [13]:
# Perform feature engineering on the dataset
def feature_engineering(dataset, aug):
    if (aug):
        dataset['los'] = dataset.apply(get_len_aug, axis=1)
    else:
        dataset['los'] = dataset.apply(get_len, axis=1)

    # bow_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 1))
    # col_name = ""
    # if aug:
    #     col_name = 'text'
    # else:
    #     col_name = 'Text'
    # text_col = dataset[col_name]
    # new_col = bow_vectorizer.fit_transform(text_col)
    # dataset['bowvec'] = new_col
    return dataset

In [16]:
train_original = pd.read_csv('dataset/fulltrain.csv')
train_augment = pd.read_csv('dataset/merged_final_df_with_topics_new.csv')

bow_vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 1))
X_ori = bow_vectorizer.fit_transform(train_original['Text'])
X_aug = bow_vectorizer.fit_transform(train_augment['text'])

y_ori = train_original['Label']
y_aug = train_augment['label']

X_train_ori, X_test_ori, y_train_ori, y_test_ori = train_test_split(X_ori, y_ori, test_size=0.20, random_state=42)
X_train_aug, X_test_aug, y_train_aug, y_test_aug = train_test_split(X_aug, y_aug, test_size=0.20, random_state=42)
train_augment[:10]

,label,text,has_swear_word,severity,processed_text,topic
0,1,"A little less than a decade ago, hockey fans w...",False,0.0,"['little', 'less', 'decade', 'ago', 'hockey', ...",0
1,1,The writers of the HBO series The Sopranos too...,False,0.0,"['writers', 'hbo', 'series', 'sopranos', 'took...",4
2,1,Despite claims from the TV news outlet to offe...,False,0.0,"['despite', 'claims', 'tv', 'news', 'outlet', ...",4
3,1,After receiving 'subpar' service and experienc...,False,0.0,"['receiving', 'subpar', 'service', 'experienci...",0
4,1,After watching his beloved Seattle Mariners pr...,False,0.0,"['watching', 'beloved', 'seattle', 'mariners',...",0
5,1,"At a cafeteria-table press conference Monday, ...",False,0.0,"['cafeteriatable', 'press', 'conference', 'mon...",0
6,1,Stunned shock and dismay were just a few of th...,False,0.0,"['stunned', 'shock', 'dismay', 'reactions', 'b...",0
7,1,"Speaking with reporters before a game Monday, ...",True,1.0,"['speaking', 'reporters', 'game', 'monday', 'l...",0
8,1,Sports journalists and television crews were p...,False,0.0,"['sports', 'journalists', 'television', 'crews...",0
9,1,"SALEM, VAF;or the eighth straight world-histor...",False,0.0,"['salem', 'vafor', 'eighth', 'straight', 'worl...",0


In [8]:
#Run this if you need to modify X_train again for some reason
# X_train = train_augment['topic']
print(X_train_ori)

  (0, 226571)	2
  (0, 176601)	4
  (0, 648)	1
  (0, 132005)	2
  (0, 226639)	4
  (0, 94016)	2
  (0, 121363)	1
  (0, 49114)	1
  (0, 209800)	1
  (0, 122141)	1
  (0, 7637)	1
  (0, 153927)	4
  (0, 63496)	1
  (0, 120226)	2
  (0, 60409)	1
  (0, 187508)	1
  (0, 206962)	1
  (0, 10164)	1
  (0, 129868)	7
  (0, 224254)	8
  (0, 47632)	3
  (0, 61177)	1
  (0, 5198)	1
  (0, 84597)	1
  (0, 120527)	1
  :	:
  (39082, 200939)	1
  (39082, 186978)	1
  (39082, 197711)	1
  (39082, 63949)	1
  (39082, 160896)	1
  (39082, 77275)	1
  (39082, 145481)	1
  (39082, 49004)	1
  (39082, 165042)	1
  (39082, 13028)	1
  (39082, 137240)	1
  (39082, 9863)	2
  (39082, 148432)	2
  (39082, 162375)	1
  (39082, 207463)	1
  (39082, 13379)	1
  (39082, 78414)	1
  (39082, 106773)	4
  (39082, 27351)	1
  (39082, 111887)	1
  (39082, 75879)	1
  (39082, 16300)	1
  (39082, 9580)	2
  (39082, 15976)	2
  (39082, 106792)	1


In [108]:
# model = GradientBoostingClassifier() # not xgboost, but just to test that a model works
# X_train_formatted = np.array(X_train).reshape(-1, 1)
# # X_train_formatted = np.array(X_train)
# model.fit(X_train, y_train)
# y_pred = model.predict(X_test)
# f1_score(y_test, y_pred, average='macro')

In [17]:
xgb_classifier = xgb.XGBClassifier(n_estimators=100, objective='binary:logistic', tree_method='hist', eta=0.1, max_depth=3, enable_categorical=True)

# The label encoder is necessary as XGBClassifier expects labels [0,1,2,3] but we have [1,2,3,4]
le = LabelEncoder()
y_train_ori = le.fit_transform(y_train_ori)
y_test_ori = le.fit_transform(y_test_ori)
y_train_aug = le.fit_transform(y_train_aug)
y_test_aug = le.fit_transform(y_test_aug)

# Original dataset
xgb_classifier.fit(X_train_ori, y_train_ori)
y_pred_ori = xgb_classifier.predict(X_test_ori)
print('Original dataset:\n')
print(classification_report(y_test_ori, y_pred_ori))

# Augmented dataset
xgb_classifier.fit(X_train_aug, y_train_aug)
y_pred_aug = xgb_classifier.predict(X_test_aug)
print('Augmented dataset:\n')
print(classification_report(y_test_aug, y_pred_aug))

Original dataset:

              precision    recall  f1-score   support

           0       0.86      0.89      0.87      2793
           1       0.92      0.84      0.88      1371
           2       0.88      0.97      0.92      3587
           3       0.92      0.77      0.83      2020

    accuracy                           0.89      9771
   macro avg       0.89      0.87      0.88      9771
weighted avg       0.89      0.89      0.88      9771

Augmented dataset:

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      2801
           1       0.90      0.87      0.88      2793
           2       0.90      0.94      0.92      3508
           3       0.85      0.81      0.83      2857

    accuracy                           0.88     11959
   macro avg       0.87      0.87      0.87     11959
weighted avg       0.88      0.88      0.88     11959



### Notes

XGBoost classifier (Sklearn version)
As a baseline for the "random" f1 score for the original dataset, I trained the XGBoost model on an X_train that was just the length of the text string. This f1 score turned out to be 0.06729. This is expected, and it just means that any meaningful features will produce an F1 score higher than this.<br>
Doing the same for the augmented dataset yields an F1 score of 0.1285. This improvement does not necessarily mean that the augmented dataset is "better", but rather that this is the base that any meaningful feature needs to beat.

We tried converting each of the text into a bag of words vector and training the XGB classifier on it. The f1 score obtained was 0.8765 for the original dataset, with an accuracy of 0.8851 and precision of 0.8935. Doing the same for the augmented dataset, the f1 score obtained was 0.8731, with accuracy 0.8759 and precision 0.8742. Although the model seems to do poorer on the augmented dataset, the discrepancy is minimal.<br>
Here, I think it is safe to conclude that when it comes to this particular feature, adding the new rows to the dataset does not affect the performance of the XGBoost classifier model.